In [199]:
#Import Packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from dotenv import load_dotenv
import os
import time

In [200]:
dashboardUrl = "https://lms.lpuonline.com/"

In [201]:
#Load the web driver and get the url
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get(dashboardUrl)

In [202]:
load_dotenv()
username = os.environ.get("registrationId")
password = os.environ.get("password")
print(type(username), type(password))

<class 'str'> <class 'str'>


In [203]:
usernameInput = driver.find_element(By.ID, "username")
passwordInput = driver.find_element(By.ID, "Password")
loginButton = driver.find_element(By.ID, "LoginButton")

In [204]:
usernameInput.send_keys(username)
passwordInput.send_keys(password)
loginButton.click()

In [205]:
totalSubjects = driver.find_element(By.ID, "crprogress").find_elements(By.CLASS_NAME, "prog-card").__len__()

In [206]:
def lectureHandler(inPlay = False):
    lectures = (driver.find_element(By.CLASS_NAME, "inPlay") if inPlay else driver.find_element(By.CLASS_NAME, "unit").find_element(By.CLASS_NAME, "collapse")).find_elements(By.CLASS_NAME, "videoli")
    for nextLectureIndex in range(lectures.__len__()):
        print("nextLectureIndex ", nextLectureIndex, lectures.__len__())
        try:
            nextLecture =  (driver.find_element(By.CLASS_NAME, "inPlay") if inPlay else driver.find_element(By.CLASS_NAME, "unit").find_element(By.CLASS_NAME, "collapse")).find_elements(By.CLASS_NAME, "videoli")[nextLectureIndex]
        except: 
            nextLecture = driver.find_element(By.CLASS_NAME, "inPlay").find_elements(By.CLASS_NAME, "videoli")[nextLectureIndex]
        nextLecture.find_element(By.CLASS_NAME, "cc-a").click()
        time.sleep(5)
        driver.execute_script(open("./videoHandler.js").read())
        time.sleep(10)

In [207]:
for currentSubjectIndex in range(totalSubjects):
    allSubjects = driver.find_element(By.ID, "crprogress").find_elements(By.CLASS_NAME, "prog-card")
    allSubjects[currentSubjectIndex].find_element(By.TAG_NAME, 'a').click()
    units = driver.find_element(By.CLASS_NAME, "unit").find_elements(By.CLASS_NAME, "collapse")
    lectureHandler(True)
    nextUnitIndex = 0
    while nextUnitIndex < units.__len__():
        print("nextUnitIndex ", nextUnitIndex)
        driver.find_element(By.CLASS_NAME, "unit").find_element(By.CLASS_NAME, "lecture-title").click()
        time.sleep(5)
        lectureHandler()
        nextUnitIndex = nextUnitIndex + 1
    driver.back()
print("All Good :D")

nextLectureIndex  0 3
nextLectureIndex  1 3
nextLectureIndex  2 3
nextLectureIndex  0 1
nextLectureIndex  0 2
nextLectureIndex  1 2
nextLectureIndex  0 2
nextLectureIndex  1 2
nextUnitIndex  0
nextLectureIndex  0 1
nextUnitIndex  1
nextLectureIndex  0 4
nextLectureIndex  1 4
nextLectureIndex  2 4
nextLectureIndex  3 4
nextUnitIndex  2
nextLectureIndex  0 5
nextLectureIndex  1 5
nextLectureIndex  2 5
nextLectureIndex  3 5
nextLectureIndex  4 5
nextUnitIndex  3
nextLectureIndex  0 5
nextLectureIndex  1 5
nextLectureIndex  2 5
nextLectureIndex  3 5
nextLectureIndex  4 5
nextUnitIndex  4
nextLectureIndex  0 1
nextUnitIndex  5
nextLectureIndex  0 3
nextLectureIndex  1 3
nextLectureIndex  2 3
nextUnitIndex  6
nextLectureIndex  0 2
nextLectureIndex  1 2
nextUnitIndex  7
nextLectureIndex  0 2
nextLectureIndex  1 2
nextUnitIndex  8
nextLectureIndex  0 2
nextLectureIndex  1 2
nextUnitIndex  9
nextLectureIndex  0 2
nextLectureIndex  1 2
nextUnitIndex  10
nextLectureIndex  0 2
nextLectureIndex  1 

IndexError: list index out of range